# Preparing the data

## Importing libraries, downloading the model

In [34]:
import pandas
import numpy
import spacy
import numpy
import sys
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

print(pandas.__version__)
print(numpy.__version__)

ImportError: cannot import name 'predict' from 'functions' (/content/functions.py)

In [ ]:
# Small Russian model for quick test purposes:
#!python -m spacy download ru_core_news_sm
#nlp = spacy.load('ru_core_news_sm')

# Large Russian model:
!python -m spacy download ru_core_news_lg
nlp = spacy.load('ru_core_news_lg')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 806.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## Making lists and doc objects from csv files

In [ ]:
# Let's load the training data from a csv file
train_set = pandas.read_csv('./train_data.csv', encoding='utf-8')
train_set.head()

,text,author
0,"Но каково же было мое изумление, когда Наташа ...",Dostoevsky
1,— закричали в толпе.. — Давай совет кошевой!,Gogol
2,"И всё, бывало, извиняется.",Chekhov
3,Живу-ут!.,Chekhov
4,Там воду освятим: они скорее выздоровеют; и я ...,Tolstoy


In [ ]:
test_set = pandas.read_csv('./test_data.csv', encoding='utf-8')
test_set.head()

,text,author
0,"""Фома Фомич, говорю, разве это возможное дело?",Dostoevsky
1,Пора бы уже домой.,Chekhov
2,"А казаки все до одного прощались, зная, что мн...",Gogol
3,"Вдруг слезы градом у обоих из глаз, дрогнули р...",Dostoevsky
4,Но художник видел в этом нежном личике одну то...,Gogol


In [ ]:
train_sentences = train_set['text'].to_list()
train_authors = train_set['author'].to_list()

test_sentences = test_set['text'].to_list()
test_authors = test_set['author'].to_list()

print(len(train_authors), len(test_authors))

10000 1000


## Creating a matrix with zero vectors for each review (in training set and test set)

In [ ]:
entity_types = ['PER', 'LOC', 'ORG']

# We are creating a matrix with zero vectors for each review (in training set and test set)
train_features_matrix = numpy.zeros((len(train_sentences), len(entity_types)))
print(train_features_matrix.shape)

test_features_matrix = numpy.zeros((len(test_sentences), len(entity_types)))
print(test_features_matrix.shape)

(10000, 3)
(1000, 3)


In [ ]:
train_doc_sentences = nlp.pipe(train_sentences)
test_doc_sentences = nlp.pipe(test_sentences)

# Modifying the feature vectors

## Visualisation tests

Visualisation test:

In [ ]:
train_doc_sentences = nlp.pipe(train_sentences)

counter = 0

for sentence, author in zip(train_doc_sentences, train_authors):
    print(sentence)
    NEs_in_sentence = [entity.label_ for entity in sentence.ents]
    print(NEs_in_sentence)
    for entity_type in entity_types:
      if entity_type in NEs_in_sentence:
        print(entity_type)

    counter +=1
    print('The author is: ', author)
    print()
    if counter == 10:
        break

Но каково же было мое изумление, когда Наташа с первых же слов остановила меня и сказала, что нечего ее утешать, что она уже пять дней, как знает про это..     – Боже мой!
['PER']
PER
The author is:  Dostoevsky

— закричали в толпе.. — Давай совет кошевой!
[]
The author is:  Gogol

И всё, бывало, извиняется.
[]
The author is:  Chekhov

Живу-ут!.
[]
The author is:  Chekhov

Там воду освятим: они скорее выздоровеют; и я теперь здоров: у меня болел глаз, а теперь смотрю в оба»..      — А мне говорили военные люди, — сказал Пьер, — что в городе никак нельзя сражаться и что позиция….
['PER']
PER
The author is:  Tolstoy

Ее-то огромное состояние у него осталось теперь, а его собственное, родовое, перешло меньшому брату, князю Ивану, который теперь обер-гоф-кафермейстер (он назвал что-то в этом роде) и был министром.. .
['PER']
PER
The author is:  Tolstoy

— сказал он.
[]
The author is:  Tolstoy

Он вступил на площадь не без какой-то невольной боязни, точно как будто сердце его предчувствовал

Visualisation Test 2:

In [ ]:
# Process a text
train_features_matrix = numpy.zeros((len(train_sentences), 3))
train_doc_sentences = nlp.pipe(train_sentences)

counter = 0
# loop over each review, label and feature vector at the same time (zip)
for sentence, author, feature_vector in zip(train_doc_sentences, train_authors, train_features_matrix):
    print('Author:', author)
    print(sentence)
    NEs_in_sentence = [entity.label_ for entity in sentence.ents]
    #print(tokens_list)
    for entity_type in entity_types:
      if entity_type in NEs_in_sentence:
        entity_id = entity_types.index(entity_type)
        print(entity_type)
        print(entity_id)
        feature_vector[entity_id] = 1
        print(feature_vector)
    print()
    counter +=1
    if counter == 10:
        break

Author: Dostoevsky
Но каково же было мое изумление, когда Наташа с первых же слов остановила меня и сказала, что нечего ее утешать, что она уже пять дней, как знает про это..     – Боже мой!
PER
0
[1. 0. 0.]

Author: Gogol
— закричали в толпе.. — Давай совет кошевой!

Author: Chekhov
И всё, бывало, извиняется.

Author: Chekhov
Живу-ут!.

Author: Tolstoy
Там воду освятим: они скорее выздоровеют; и я теперь здоров: у меня болел глаз, а теперь смотрю в оба»..      — А мне говорили военные люди, — сказал Пьер, — что в городе никак нельзя сражаться и что позиция….
PER
0
[1. 0. 0.]

Author: Tolstoy
Ее-то огромное состояние у него осталось теперь, а его собственное, родовое, перешло меньшому брату, князю Ивану, который теперь обер-гоф-кафермейстер (он назвал что-то в этом роде) и был министром.. .
PER
0
[1. 0. 0.]

Author: Tolstoy
— сказал он.

Author: Gogol
Он вступил на площадь не без какой-то невольной боязни, точно как будто сердце его предчувствовало что-то недоброе.

Author: Chekhov
Гос

## Writing a function for feature vector modification

In [ ]:
def modify_feature_vectors(doc_sentences, features_matrix):
  for sentence, feature_vector in zip(doc_sentences, features_matrix):
      NEs_in_sentence = [entity.label_ for entity in sentence.ents]
      for entity_type in entity_types:
        if entity_type in NEs_in_sentence:
          entity_id = entity_types.index(entity_type)
          feature_vector[entity_id] = 1
  return features_matrix

## Modify the feature vector

In [ ]:
train_features_matrix = numpy.zeros((len(train_sentences), 3))
train_doc_sentences = nlp.pipe(train_sentences)

train_features_matrix_final = modify_feature_vectors(train_doc_sentences, train_features_matrix)

# Training

In [ ]:
lr = LogisticRegression()

lr.fit(train_features_matrix_final, train_authors)

print(lr.classes_)

['Chekhov' 'Dostoevsky' 'Gogol' 'Tolstoy']


# Modify the test set feature vectors

In [ ]:
test_doc_sentences = nlp.pipe(test_sentences)
test_features_matrix = numpy.zeros((len(test_sentences), 3))

test_features_matrix_final = modify_feature_vectors(test_doc_sentences, test_features_matrix)

# Making predictions

In [ ]:
def predict(i, features=None):
    print(test_sentences[i])
    # print the features of the index
    print(test_features_matrix_final[i])
    # print all entity types
    if features:
      print(features)
    # print the correct label of the index
    print(test_authors[i])

    print()
    print("Prediction:")
    # print the prediction for the features of this index
    print(lr.predict([test_features_matrix_final[i]]))
    # print the probabilities for each label predictions
    print(lr.predict_proba([test_features_matrix_final[i]]))
    print("--------------------------------------------------------")
    print()

In [ ]:
predict(0)
predict(5, features=entity_types)
predict(25, features=entity_types)

"Фома Фомич, говорю, разве это возможное дело?
[1. 0. 0.]
Dostoevsky

Prediction:
['Chekhov']
[[0.28198236 0.22214749 0.22817444 0.26769571]]
--------------------------------------------------------

Когда я в Берлине получил оттуда несколько маленьких писем, которые они уже успели мне написать, то тут только я и понял, как их любил.
[0. 1. 0.]
['PER', 'LOC', 'ORG']
Dostoevsky

Prediction:
['Tolstoy']
[[0.1879233  0.18556256 0.27324013 0.35327401]]
--------------------------------------------------------

Так поедешь?
[0. 0. 0.]
['PER', 'LOC', 'ORG']
Tolstoy

Prediction:
['Dostoevsky']
[[0.24212508 0.26604665 0.25930398 0.23252428]]
--------------------------------------------------------



In [ ]:
test_predictions_ner = lr.predict(test_features_matrix_final)

In [ ]:
def predict2(test_predictions, test_authors, m=0, n=10):
  for prediction, author in zip(test_predictions[m:n], test_authors[m:n]):
    if prediction == author:
        result_is = "Correct"
    else:
        result_is = "Incorrect"
    print(f"{prediction} ({result_is}:{author})")

In [ ]:
predict2(test_predictions_ner, test_authors, m=0, n=5)

Chekhov (Incorrect:Dostoevsky)
Dostoevsky (Incorrect:Chekhov)
Dostoevsky (Incorrect:Gogol)
Dostoevsky (Correct:Dostoevsky)
Dostoevsky (Incorrect:Gogol)


# Analysing the model

In [ ]:
def model_analysis(lr_model):
  for label, coefs, intercept in zip(lr_model.classes_, lr_model.coef_, lr_model.intercept_):
    print(label)
    for t, c in zip(entity_types, coefs):
        print(t, c)
    print("INTERCEPT:", intercept)
    print()

In [ ]:
model_analysis(lr)

Chekhov
PER 0.1561336751358554
LOC -0.21764533083370355
ORG -0.23611650129519218
INTERCEPT: -0.030575998134737735

Dostoevsky
PER -0.17658652419566542
LOC -0.32450439653856056
ORG 0.08049482072985621
INTERCEPT: 0.06364122153367807

Gogol
PER -0.12414701627264657
LOC 0.08812516959084483
ORG -0.43733717051896115
INTERCEPT: 0.037970579292936395

Tolstoy
PER 0.14459986533245467
LOC 0.45402455778142037
ORG 0.5929588510842979
INTERCEPT: -0.07103580269125283



# Saving the model

In [ ]:
# Save to file in the current working directory
pkl_filename = "logreg_NER.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(lr, file)

# Evaluating the model

## Dummy model

In [ ]:
dummy_predictions = ['Dostoevsky'] * len(test_sentences)
print(classification_report(test_authors, dummy_predictions))

              precision    recall  f1-score   support

     Chekhov       0.00      0.00      0.00       250
  Dostoevsky       0.25      1.00      0.40       250
       Gogol       0.00      0.00      0.00       250
     Tolstoy       0.00      0.00      0.00       250

    accuracy                           0.25      1000
   macro avg       0.06      0.25      0.10      1000
weighted avg       0.06      0.25      0.10      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## NER Model

In [ ]:
print(classification_report(test_authors, test_predictions_ner))

              precision    recall  f1-score   support

     Chekhov       0.30      0.30      0.30       250
  Dostoevsky       0.24      0.68      0.35       250
       Gogol       0.00      0.00      0.00       250
     Tolstoy       0.36      0.06      0.11       250

    accuracy                           0.26      1000
   macro avg       0.23      0.26      0.19      1000
weighted avg       0.23      0.26      0.19      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
